In [108]:
import plotly
import pandas as pd
#import plotly.plotly as py
import plotly.offline as pyo
from plotly.graph_objs import *
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

In [109]:
user_df=pd.read_csv("user_df.csv")
df=pd.read_csv('latest_df.csv')


In [94]:
def risk_analizi(df,user_df,user=28):
    df2=df.loc[df.UserId==user]
    
    df_merged = pd.merge(left=df2, right=user_df, how='left', left_on=('UserId'),right_on=('UserId')) ## ,how=left
    
    df4=df_merged.loc[:,['UserId','Age','Temperature','ProcessDate']]

    df4['Rate'] = 0
    ### Temps
    df4.loc[(df4['Temperature']>=38 ) , 'Rate'] = 1.0
    df4.loc[(df4['Temperature']>=37.5) & (df4['Temperature']<38), 'Rate'] = 0.8
    df4.loc[(df4['Temperature']>=37) & (df4['Temperature']<37.5), 'Rate'] = 0.6
    df4.loc[(df4['Temperature']>=36.5) & (df4['Temperature']<37), 'Rate'] = 0.4
    df4.loc[(df4['Temperature']>=36) & (df4['Temperature']<36.5), 'Rate'] = 0.2
    df4.loc[(df4['Temperature']>=35.5) & (df4['Temperature']<36), 'Rate'] = 0.15
    df4.loc[(df4['Temperature']>=35) & (df4['Temperature']<35.5), 'Rate'] = 0.10
    df4.loc[(df4['Temperature']>=34.5) & (df4['Temperature']<35), 'Rate'] = 0.05


    ### Ages
    df4.loc[(df4['Age']>=60),'Rate']+=0.6
    df4.loc[(df4['Age']<15,'Rate')]+=0.4
    df4.loc[(df4['Age']<60)&(df4['Age']>=50),'Rate']+=0.3
    return df4


In [66]:
# LSTM for international airline passengers problem with time step regression framing
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [106]:
def Risk_Tahmini(df4,size=0.7):
    df4['ProcessDate'] = pd.to_datetime(df4['ProcessDate'] )
    df5=df4.loc[:,['ProcessDate','Rate']]
    df5.head()
    df5=df5.tail(100)
    df6=df5.set_index('ProcessDate')
    rolling_mean = df6.rolling(window=2).mean()
    # load the dataset
    rolling_mean=rolling_mean.dropna()
    dataset = rolling_mean.values
    dataset = dataset.astype('float32')
    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return numpy.array(dataX), numpy.array(dataY)

    # fix random seed for reproducibility
    numpy.random.seed(7)

    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    # split into train and test sets
    train_size = int(len(dataset) * size)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    # reshape into X=t and Y=t+1
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    # reshape input to be [samples, time steps, features]
    trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
    testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))
    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    # shift train predictions for plotting
    trainPredictPlot = numpy.empty_like(dataset)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
    # plot baseline and predictions
    df = pd.DataFrame(trainPredictPlot,columns=["train"])
    df2= pd.DataFrame(testPredictPlot,columns=["test"])
    return df,df2,rolling_mean

In [110]:
df_=df.copy()
user_df_=user_df.copy()

In [111]:

df4=risk_analizi(df_,user_df_,user=28)   

df,df2,rolling_mean=Risk_Tahmini(df4,size=0.7)

## Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=rolling_mean.index, y=df['train'],
                         marker_color='indianred',
                    mode='lines+markers',
                    name='Gercek Risk Degeri'))

fig.add_trace(go.Scatter(x=rolling_mean.index,y=df2['test'],
        name='Risk Tahmini',
        marker_color='lightsalmon',mode='lines+markers'))

fig.update_yaxes(range=[0.5,1])

fig.show()

Epoch 1/100
 - 0s - loss: 0.2507
Epoch 2/100
 - 0s - loss: 0.0993
Epoch 3/100
 - 0s - loss: 0.0466
Epoch 4/100
 - 0s - loss: 0.0403
Epoch 5/100
 - 0s - loss: 0.0395
Epoch 6/100
 - 0s - loss: 0.0396
Epoch 7/100
 - 0s - loss: 0.0393
Epoch 8/100
 - 0s - loss: 0.0392
Epoch 9/100
 - 0s - loss: 0.0386
Epoch 10/100
 - 0s - loss: 0.0396
Epoch 11/100
 - 0s - loss: 0.0380
Epoch 12/100
 - 0s - loss: 0.0375
Epoch 13/100
 - 0s - loss: 0.0381
Epoch 14/100
 - 0s - loss: 0.0374
Epoch 15/100
 - 0s - loss: 0.0375
Epoch 16/100
 - 0s - loss: 0.0374
Epoch 17/100
 - 0s - loss: 0.0371
Epoch 18/100
 - 0s - loss: 0.0374
Epoch 19/100
 - 0s - loss: 0.0370
Epoch 20/100
 - 0s - loss: 0.0368
Epoch 21/100
 - 0s - loss: 0.0365
Epoch 22/100
 - 0s - loss: 0.0365
Epoch 23/100
 - 0s - loss: 0.0361
Epoch 24/100
 - 0s - loss: 0.0358
Epoch 25/100
 - 0s - loss: 0.0362
Epoch 26/100
 - 0s - loss: 0.0358
Epoch 27/100
 - 0s - loss: 0.0360
Epoch 28/100
 - 0s - loss: 0.0355
Epoch 29/100
 - 0s - loss: 0.0355
Epoch 30/100
 - 0s - lo